In [5]:
# Some Imports You Might Need
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model as lm
from sklearn import ensemble

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

# Extract Dataset
with zipfile.ZipFile('cook_county_contest_data.zip') as item:
    item.extractall()
from sklearn import linear_model as lm
import torch
pd.set_option('display.max_columns', None)
from sklearn import datasets
import matplotlib.pyplot as plt
import tensorflow as tf
import torch.nn as nn
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib 

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from sklearn.naive_bayes import MultinomialNB     # 从sklean.naive_bayes里导入朴素贝叶斯模型
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf
#print('Tensorflow Version: {}'.format(tf.__version__))
import pandas as pd
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'# 只显示 Error
import warnings
warnings.filterwarnings("ignore")
from random import seed

In [2]:
from sklearn.preprocessing import OneHotEncoder

def add_total_bedrooms(data):
    """
    Input:
      data (data frame): a data frame containing at least the Description column.
    """
    with_rooms = data.copy()
    pattern = r'(\d+)(.\d+)? of which (are|is) (bedrooms|bedroom)'
    with_rooms['Bedrooms'] = with_rooms['Description'].apply(lambda x: int(re.findall(pattern, x)[0][0]) if len(re.findall(pattern, x))>0 else np.nan)
    with_rooms.fillna(0)
    return with_rooms

def ohe_roof_material(data):
    """
    One-hot-encodes roof material.  New columns are of the form 0x_QUALITY.
    """
    ohe = OneHotEncoder()
    new_columns = pd.DataFrame(data=ohe.fit_transform(data[['Roof Material']]).todense(), 
                              columns=ohe.get_feature_names(),
                             index = data.index)
    return data.join(new_columns)

def substitute_roof_material(data):
    """
    Input:
      data (data frame): a data frame containing a 'Roof Material' column.  Its values
                         should be limited to those found in the codebook
    Output:
      data frame identical to the input except with a refactored 'Roof Material' column
    """
    dictionary = {1:'Shingle/Asphalt', 2:'Tar&Gravel', 3:'Slate', 4:'Shake', 5:'Tile', 6:'Other'}
    data['Roof Material'] = data['Roof Material'].replace(dictionary)
    return data
  
def process_data_gm(data, pipeline_functions, prediction_col):
    """Process the data for a guided model."""
    for function, arguments, keyword_arguments in pipeline_functions:
        if keyword_arguments and (not arguments):
            data = data.pipe(function, **keyword_arguments)
        elif (not keyword_arguments) and (arguments):
            data = data.pipe(function, *arguments)
        else:
            data = data.pipe(function)
    X = data.drop(columns=[prediction_col]).to_numpy()
    y = data.loc[:, prediction_col].to_numpy()
    return X, y

def add_in_expensive_neighborhood(data, neighborhoods):
    """
    Input:
      data (data frame): a data frame containing a 'Neighborhood Code' column with values
        found in the codebook
      neighborhoods (list of strings): strings should be the names of neighborhoods
        pre-identified as rich
    Output:
      data frame identical to the input with the addition of a binary
      in_rich_neighborhood column
    """
    data['in_expensive_neighborhood'] = data['Neighborhood Code'].isin(neighborhoods).astype(int)
    return data

def find_expensive_neighborhoods(data, n=3, metric=np.median):
    """
    Input:
      data (data frame): should contain at least a string-valued Neighborhood
        and a numeric 'Sale Price' column
      n (int): the number of top values desired
      metric (function): function used for aggregating the data in each neighborhood.
        for example, np.median for median prices
    
    Output:
      a list of the top n richest neighborhoods as measured by the metric function
    """
    neighborhoods = data.groupby('Neighborhood Code').agg({'Log Sale Price':metric}).sort_values('Log Sale Price',ascending=False).head(n).index
    
    # This makes sure the final list contains the generic int type used in Python3, not specific ones used in numpy.
    return [int(code) for code in neighborhoods]

def select_columns(data, *columns):
    """Select only columns passed as arguments."""
    return data.loc[:, columns]

def rmse(predicted, actual):
    return np.sqrt(np.mean((actual - predicted)**2))

In [3]:
def process_data_fm(data):
    X = data
    X['Log Building Square Feet'] = np.log(X['Building Square Feet'])
    X = add_total_bedrooms(X)
    X = substitute_roof_material(X)
    X = ohe_roof_material(X)
    X = select_columns(X, 'Bedrooms','Property Class', 'Log Building Square Feet', 'x0_Other', 'x0_Shake', 'x0_Shingle/Asphalt',
                       'x0_Slate', 'x0_Tar&Gravel', 'x0_Tile','Age', 'Apartments')
    return X

In [4]:
train_data = pd.read_csv('cook_county_contest_data/cook_county_contest_train.csv')
y_train = np.log(train_data['Sale Price'])
X_train = process_data_fm(train_data)
size, features = X_train.shape

## 3-Layer MLP, 50 epoches

In [9]:
inputs = tf.keras.Input(shape=(features, ))
hidden = tf.keras.layers.Dense(64, activation='relu')(inputs)
hidden = tf.keras.layers.Dense(64, activation='relu')(hidden)
hidden = tf.keras.layers.Dense(64, activation='relu')(hidden)
outputs = tf.keras.layers.Dense(1, activation='linear')(hidden)

tf_model = tf.keras.Model(inputs, outputs)

In [11]:
# train the model
tf_model.compile(
    optimizer = 'adam',
    loss = 'mse'
)

history = tf_model.fit(
    X_train,
    y_train,
    validation_split=0.12,
    batch_size=32,
    epochs=50
)

Epoch 1/50
3801/3801 [==============================] - 11s 3ms/step - loss: 0.4637 - val_loss: 0.4412
Epoch 2/50
3801/3801 [==============================] - 11s 3ms/step - loss: 0.4630 - val_loss: 0.4487
Epoch 3/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4596 - val_loss: 0.4488
Epoch 4/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4625 - val_loss: 0.4456
Epoch 5/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4608 - val_loss: 0.4421
Epoch 6/50
3801/3801 [==============================] - 9s 2ms/step - loss: 0.4574 - val_loss: 0.4508
Epoch 7/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4595 - val_loss: 0.4434
Epoch 8/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4581 - val_loss: 0.4479
Epoch 9/50
3801/3801 [==============================] - 10s 3ms/step - loss: 0.4575 - val_loss: 0.4421
Epoch 10/50
3801/3801 [==============================] - 10s 3ms/step - lo

In [12]:
test_data = pd.read_csv('cook_county_contest_test.csv')
X_test = process_data_fm(test_data)
tf_predicted = tf_model.predict(X_test)


480/480 [==============================] - 1s 1ms/step


In [13]:
answer = pd.read_csv("saleprice.csv")
answer = answer.drop("Unnamed: 0",axis=1)
answer['log_price'] = np.log(answer['Sale Price'])
answer['log_tf_predicted'] = tf_predicted
answer['tf_predicted'] = np.exp(tf_predicted)

In [14]:
rmse(answer['Sale Price'],answer['tf_predicted'])

133496.70923810574

## Random Forest

In [28]:
rf_model = ensemble.RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [30]:
rf_predicted = rf_model.predict(X_test)
answer['rf_predicted'] = rf_predicted
answer['rf_predicted'] = np.exp(rf_predicted)
rmse(answer['Sale Price'],answer['rf_predicted'])

136399.4385239601

In [ ]:
rmse(answer['log_price'], answer['lr_predicted'])

## Simple Linear SVR

In [19]:
from sklearn.svm import LinearSVR

In [24]:
eps = 5
svr = LinearSVR(epsilon=eps, C=0.01, fit_intercept=True)
svr.fit(X_train, y_train)

LinearSVR(C=0.01, epsilon=5)

In [27]:
svr_predicted = svr.predict(X_test)
answer['log_svr_predicted'] = svr_predicted
answer['svr_predicted'] = np.exp(svr_predicted)
rmse(answer['Sale Price'],answer['svr_predicted'])

267355.8661293371

## LSTM

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import preprocessing 

In [45]:
X_train.shape

(138217, 11)

In [50]:
lstm = Sequential()
lstm.add(LSTM(32, return_sequences = True))
lstm.add(LSTM(16))
lstm.add(Dense(1))
lstm.add(Activation('linear'))

In [51]:
lstm.compile(loss='mean_squared_error', optimizer='adagrad') # Try SGD, adam, adagrad and compare
lstm.fit(X_train, y_train, epochs=5, batch_size=1, verbose=2)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\zihen\AppData\Roaming\Python\Python39\site-packages\keras\engine\